In [2]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
import tensorflow as tf
import mlflow
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,accuracy_score,confusion_matrix,classification_report
from mlflow.models import infer_signature
from sklearn.linear_model import LogisticRegression
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn import datasets
from mlflow.models.signature import infer_signature
import mlflow.sklearn

In [8]:
import tensorflow as tf
import numpy as np

def load_dataset(data_dir, img_size=(150,150), batch=32):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=img_size,
        batch_size=batch
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=img_size,
        batch_size=batch
    )
    return train_ds, val_ds

def preprocess_image(img_path, img_size=(150,150)):
    img = tf.keras.utils.load_img(img_path, target_size=img_size)
    img = tf.keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0) / 255.0
    return img


In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

import tensorflow as tf
from tensorflow.keras import layers, models
import mlflow.tensorflow

import os

data_dir = "data"
mlflow.set_experiment("Eye Disease Detection")

def build_model():
    model = models.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.Rescaling(1./255),

        layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        layers.MaxPooling2D(),
        layers.BatchNormalization(),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.BatchNormalization(),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.BatchNormalization(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

with mlflow.start_run():
    train_ds, val_ds = load_dataset(data_dir)
    model = build_model()

    history = model.fit(train_ds, validation_data=val_ds, epochs=10)

    mlflow.log_metric("train_accuracy", history.history['accuracy'][-1])
    mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])

    os.makedirs("models", exist_ok=True)
    model.save("models/eye_disease_model.h5")
    mlflow.log_artifact("models/eye_disease_model.h5")

print("✅ Training completed & model saved!")


In [12]:
import tensorflow as tf
import numpy as np



model = tf.keras.models.load_model("eye_disease_model.h5")

class_names = ['Normal', 'Cataract', 'Glaucoma']  # change as needed

img_path = "eye.jpg"
img = preprocess_image(img_path)
pred = model.predict(img)

class_idx = np.argmax(pred)
confidence = pred[0][class_idx]

print(f"🩺 Prediction: {class_names[class_idx]}, Confidence: {confidence*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🩺 Prediction: Glaucoma, Confidence: 20.90%


In [1]:
import tensorflow as tf
print("TF version:", tf.__version__)

TF version: 2.20.0
